In [2]:
import numpy as np
import random,sys

In [10]:
#commencer à explorer à partir le nœud initial
def explore(path,distances,visitedCities,pheromone,cities,z):
    #enregistrer le nœud initial
    initialNode=visitedCities[z][len(path)-1]
    
    #choisissez le nœud suivant
    chooseNextNode(initialNode,distances,visitedCities,pheromone,cities,z)

#fonction de choix du nœud suivant
def chooseNextNode(initialNode,distances,visitedCities,pheromone,cities,z):
    #tableau pour enregistrer la probabilite de chaque nœud d'etre le nœud choisi
    nextNodesProba={}
    
    #tableau pour conserver la probabilite cumulee (a utiliser dans la srlection roulette)
    cumulativeSum=[]
    
    #variable qui contiendra le calcul de la somme 𝛕(nœud initial,j) *η(nœud initial,j)
    #où j est les prochaines villes non visitées
    #où 𝛕(nœud initial,j) la quantite de pheromone entre le noeud initial et j et η(nœud initial,j) est 1/distance(noeud initial,j)
    s=0
    
    c=0
    
    #itérer la liste des villes
    for i in cities:
        #si la ville est la ville initiale ou si la ville est deja visitee, passez à l'iteration suivante
        if i==initialNode or i in visitedCities[z]:
            continue
            
        #sinon calculer la probabilite de choisir cette ville
        nextNodesProba[i]=pheromone[initialNode][i]*(1/distances[initialNode][i])
        s+=pheromone[initialNode][i]*(1/distances[initialNode][i])
    for node in list(nextNodesProba.keys()):
        nextNodesProba[node]=round(nextNodesProba[node]/s,3)
     
    #trier le tableau de probabilités par ordre décroissant
    nextNodesProba=dict(sorted(nextNodesProba.items(), key=lambda item: item[1],reverse=True))
    
    
    #calcul de probabilité cumulée 
    for i in range(len(nextNodesProba)):
        j=i
        for proba in nextNodesProba.values():
            if j!=0:
                j-=1
                continue
            c+=proba
        c=round(c,2)
        cumulativeSum.append(c)
        c=0

    #choisir une valeur aléatoire entre 0 et 1    
    choose=random.random()
    
    #choisissez la ville suivante en utilisant le nombre aleatoire precedent
    chosenCity=-1
    optionCities=list(nextNodesProba.keys())
    for i in range(0,len(cumulativeSum)-1):
        #si le nombre est compris entre 2 nombres cumulés on choisit la ville de cette tranche
        if choose<=cumulativeSum[i] and choose>cumulativeSum[i+1]:
            chosenCity=optionCities[i]
            break
    if choose<=cumulativeSum[len(cumulativeSum)-1] and choose>0:
            chosenCity=optionCities[len(cumulativeSum)-1]
            
    #enregistrer cette ville comme ville visitée
    visitedCities[z].append(chosenCity)

#modification de la quantité de phéromone
def pheromoneModification(visitedCities,pheromone,cities,distances):
    #taux de vaporisation
    vaporisationPercenatge=0.1
    
    #ce tableau contiendra le nombre de fourmis qui sont passées dans les deux villes
    pheromoneToAdd={}
    
    #itere les villes visitées depuis la dernière ville visitée jusqu'au premier.
    for i in range(len(visitedCities[0])-1,-1,-1):
        if i==0:
            break
        #indice initial de la ville
        lastpos=i
        
        #index de la villes précédente
        retour=i-1
        
        for key in list(visitedCities.keys()):
            lastPath=(visitedCities[key][retour],visitedCities[key][lastpos])
            #si le chemin de ces deux villes n'est pas dans le tableau on ajoute ce tableau avec la valeur 1
            if lastPath not in pheromoneToAdd:
                pheromoneToAdd[lastPath]=1
                
            #si le chemin de ces deux villes déjà dans le tableau incrémente la valeur de 1
            else:
                pheromoneToAdd[lastPath]+=1
    for i in list(pheromoneToAdd.keys()):
        initialNode=i[1]
        for j in cities:
            if j==initialNode:
                continue
            if (j,initialNode) in (pheromoneToAdd.keys()):
                #si les deux villes en 'pheromonetoadd' on applique la formule de modification des phéromones
                pheromone[initialNode][j]=((1-vaporisationPercenatge)*pheromone[initialNode][j])+(pheromoneToAdd[(j,initialNode)]*(1/distances[initialNode][j]))
                pheromone[j][initialNode]=pheromone[initialNode][j]
            else:
                #si non on applique seulement la vaporisation
                pheromone[initialNode][j]=((1-vaporisationPercenatge)*pheromone[initialNode][j])
                pheromone[j][initialNode]=pheromone[initialNode][j]       

#fonction pour calculer la distance du chemin
def calculateDistances(path,distances):
    cost=0
    for i in range(len(path)-1):
        cost+=distances[path[i]][path[i+1]]
    cost+=distances[path[len(path)-1]][0]
    return cost

#fonction 'Tournament Selection'
def finalRoulette(paths):
    totalSum=0
    c=0
    pathsProb={}
    cumulativeSum=[]
    s=0
    for i in paths:
        totalSum+=1/i[1]
    for i in paths:
        pathsProb[tuple(i[0])]=(1/i[1])/totalSum
        s+=(1/i[1])/totalSum

    pathsProb=dict(sorted(pathsProb.items(), key=lambda item: item[1],reverse=True))
    for i in range(len(pathsProb)):
        j=i
        for proba in pathsProb.values():
            if j!=0:
                j-=1
                continue
            c+=proba
        c=round(c,2)
        cumulativeSum.append(c)
        c=0

    choose=random.random()
    chosenPath=[]
    
    optionPaths=list(pathsProb.keys())
    for i in range(0,len(cumulativeSum)-1):#roulette
        if choose<=cumulativeSum[i] and choose>cumulativeSum[i+1]:
            chosenPath=list(optionPaths[i])
            break
    if choose<=cumulativeSum[len(cumulativeSum)-1] and choose>0:
            chosenPath=list(optionPaths[len(cumulativeSum)-1])
    return chosenPath

#fonction de 'Roulette Wheel Selection'
def finalNormal(paths):
    min=10000000
    chosenPath=[]
    for i in paths:
        if i[1]<=min:
            min=i[1]
            chosenPath=list(i[0])
    return chosenPath
            

In [ ]:
nbCities=5#random.randint(6,8)
print(nbCities-1)
cities=[]
paths=[]
distances=np.zeros((nbCities-1,nbCities-1))
pheromone=np.ones((nbCities-1,nbCities-1))

for i in range(nbCities-1):
    for j in range(nbCities-1):
        if i==j:
            pheromone[i][j]=0

#nombre de fourmis
k=10#random.randint(4,10)

#remplir le tableau 'cities' avec les villes
for i in range(0,nbCities-1):
    cities.append(i)

print(cities)
print(paths)
print(k)

p = [[0,13,25,19],
     [23,0,22,10],
     [15,2,0,23],
     [14,10,1,0]]

distances=np.array(p)

# #remplir le tableau 'distances' avec des distances aléatoires entre 2 villes d'une manière à obtenir une matrice d'ajacance
# for i in range(0,nbCities-1):
#     for j in range(0,nbCities-1):
#         if i==j:
#             distances[i][j]=0
#             distances[j][i]=distances[i][j]
#             continue
#         distances[i][j]=random.randint(10,100)
#         distances[j][i]=distances[i][j]
# print(distances,'\n')

#pour chaque ville nous calculerons le chemin le plus court à partir de celle-ci
for i in cities:
    path=[]
    visitedCities={}
    test=0
    node=i
    #on lance l'algorithme avec 100 iteration 
    for j in range(0,100):
        while 1:
            #chaque fourmi démarre le processus d'exploration
            for z in range(0,k):
                #vérifier la ville si elle est visitée
                if z not in visitedCities:
                    visitedCities[z]=[i]
                explore(path,distances,visitedCities,pheromone,cities,z)
                
            #si le nombre de villes visitées est égal au nombre de villes, nous terminons le processus d'exploration
            if len(visitedCities[k-1])==nbCities-1 or test==2:
                break
                
        #modifier les pheromones
        pheromoneModification(visitedCities,pheromone,cities,distances)
        visitedCities={}
    print("")
    
    #construire le chemin à partir des valeurs de phéromones les plus élevées
    path.append(node)
    while 1:
        max=-10000.0
        for idx1 in range(len(pheromone[0])):
            if idx1 in path:
                continue
            if pheromone[node][idx1]>max:
                n,max=idx1,pheromone[node][idx1]
        if len(path)==nbCities-1:
            break
        path.append(n)
        node=n
        if len(path)!=nbCities-1:
            continue
    #calculer le cout de chemin
    cost=calculateDistances(path,distances)

    #paths contient tous les chemins choisis
    paths.append((path,cost))

#trier par ordre décroissant la liste des chemins choisis en partant de chaque ville par le coût
paths=sorted(paths,key=lambda x: x[1], reverse=True)
print(paths)

#choisissez parmi ces chemins le chemin le plus court en utilisant la fonction de 'Roulette Wheel Selection' 
chosenPathRoulette=finalRoulette(paths)

#afficher le chemin le plus court choisi par la méthode de la roulette
print("the chosen path with roulette method:",chosenPathRoulette)
print("the distance is:",calculateDistances(chosenPathRoulette,distances))

#choisissez parmi ces chemins le chemin le plus court en utilisant la fonction de 'Tournament Selection' 
chosenPathNormal=finalNormal(paths)

#afficher le chemin le plus court choisi par la méthode de 'Tournament Selection'
print("the chosen path with minimum cost method:",chosenPathNormal)
print("the distance is:",calculateDistances(chosenPathNormal,distances))

    
    
